In [ ]:
import requests
from config import API_KEY, BASE_URL

def get_corp_code(corp_name):
    # 기업 이름으로 고유 코드 가져오기
    ...

def fetch_financial_data(corp_code, year):
    url = f"{BASE_URL}fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011"  # 사업보고서
    }
    response = requests.get(url, params=params)
    return response.json()


# ROE

In [ ]:
import requests, zipfile, io
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import os

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

API_KEY = 
YEARS = [str(y) for y in range(2015, 2024)]

CorpList = [
    "삼성전자", "SK하이닉스", "LG에너지솔루션", "삼성바이오로직스", "KB금융", "현대차", "삼성전자우",
    "NAVER", "기아", "셀트리온", "HD현대중공업", "신한지주", "삼성물산", "현대모비스", "하나금융지주",
    "LG화학", "SK이노베이션", "고려아연", "크래프톤", "삼성SDI", "현대건설", "LG생활건강", "카카오",
    "SK텔레콤", "S‑오일", "LG유플러스", "포스코", "SK", "두산", "LG", "한화케미칼", "엔씨소프트",
    "롯데케미칼", "KB증권", "한국전력", "아모레퍼시픽", "KT", "KT&G", "CJ제일제당", "현대제철",
    "한온시스템", "LG전자", "두산인프라코어", "우리금융지주", "신세계", "하나투어", "CJ ENM",
    "LG이노텍", "대한항공", "SK케미칼", "SKC", "한미약품", "한국조선해양", "이마트", "GS",
    "현대중공업", "SK아이이테크놀로지", "DB하이텍", "CJ대한통운", "HMM", "한미사이언스", "오리온",
    "LG화학우", "SK머티리얼즈", "셀트리온헬스케어", "한국금융지주", "삼성에스디에스", "SK바이오사이언스",
    "하나제약", "LG디스플레이", "S-Oil우", "카카오뱅크", "한미글로벌", "포스코케미칼", "KTcs",
    "DB손해보험", "한국항공우주", "에쓰오일", "LG생활건강우", "셀트리온제약", "신한지주우",
    "SK이노베이션우", "하나금융지주우", "코스맥스", "넷마블", "CJ", "삼성엔지니어링", "KLAC",
    "CJ헬스케어", "롯데지주", "한화생명", "한화솔루션", "한국타이어앤테크놀로지", "한온시스템우",
    "현대글로비스", "S&T중공업", "삼성전기", "포스코인터내셔널", "HDC현대산업개발", "현대자동차우",
    "한화에어로스페이스"
]

def get_corp_code_dict(api_key=API_KEY):
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    res = requests.get(url, params={"crtfc_key": api_key})
    with zipfile.ZipFile(io.BytesIO(res.content)) as zf:
        zf.extractall()
    tree = ET.parse("CORPCODE.xml")
    root = tree.getroot()
    return {
        el.find("corp_name").text.strip(): el.find("corp_code").text.strip()
        for el in root.findall("list")
    }

def parse_amount(s):
    if not s or s in ["", "-"]:
        return None
    return int(s.replace(",", "").replace(" ", "").strip())

def get_financial_metrics(corp_code, year):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",
        "fs_div": "CFS"
    }
    try:
        res = requests.get(url, params=params, timeout=10).json()
    except:
        return None, None, None, None

    net_income = None
    equity = None
    sales = None
    operating_income = None

    for item in res.get("list", []):
        name = item.get("account_nm", "")
        val = parse_amount(item.get("thstrm_amount"))

        if "순이익" in name and not net_income:
            net_income = val
        elif ("자본총계" in name or "총자본" in name) and not equity:
            equity = val
        elif "영업이익" in name and not operating_income:
            operating_income = val
        elif ("매출액" in name or "수익" in name or "매출" in name) and not sales:
            sales = val

    return net_income, equity, operating_income, sales

def generate_roe_df(
    corp_name,
    save_dir_csv=r"C:\Users\1\Desktop\Dart\roe.csv",
    save_dir_png=r"C:\Users\1\Desktop\Dart\roe.plot"
):
    print(f"📊 {corp_name} ROE 계산 중...")

    corp_code_dict = get_corp_code_dict()
    corp_code = corp_code_dict.get(corp_name)
    if not corp_code:
        print(f"❌ 기업 코드 없음: {corp_name}")
        return None

    rows = []
    for year in YEARS:
        net_income, equity, _, _ = get_financial_metrics(corp_code, year)
        roe = round(net_income / equity * 100, 2) if net_income and equity else None
        rows.append({
            "연도": year,
            "당기순이익": net_income,
            "자본총계": equity,
            "ROE (%)": roe
        })

    df = pd.DataFrame(rows)

    os.makedirs(save_dir_csv, exist_ok=True)
    csv_path = os.path.join(save_dir_csv, f"{corp_name}_ROE.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ ROE CSV 저장 완료: {csv_path}")

    os.makedirs(save_dir_png, exist_ok=True)
    plt.figure(figsize=(10, 5))
    plt.plot(df["연도"], df["ROE (%)"], marker="o", label="ROE (%)", color="green")
    plt.title(f"{corp_name} ROE 추이", fontsize=14)
    plt.xlabel("연도")
    plt.ylabel("ROE (%)")
    plt.grid(True)
    plt.tight_layout()
    png_path = os.path.join(save_dir_png, f"{corp_name}_ROE.png")
    plt.savefig(png_path)
    print(f"🖼️ ROE 그래프 저장 완료: {png_path}")
    plt.close()

    return df


# 영업이익률

In [ ]:
def generate_operating_margin_df(
    corp_name,
    save_dir_csv=r"C:\Users\1\Desktop\Dart\영업이익률csv",
    save_dir_png=r"C:\Users\1\Desktop\Dart\영업이익률plot"
):
    print(f"📉 {corp_name} 영업이익률 계산 중...")

    # 기업 코드 조회
    corp_code_dict = get_corp_code_dict()
    corp_code = corp_code_dict.get(corp_name)
    if not corp_code:
        print(f"❌ 기업 코드 없음: {corp_name}")
        return None

    # 연도별 데이터 수집
    data = []
    for year in YEARS:
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": "11011",  # 사업보고서
            "fs_div": "CFS"         # 연결재무제표
        }

        try:
            res = requests.get(url, params=params, timeout=10).json()
        except:
            data.append((year, None, None))
            continue

        op_income = None
        revenue = None

        for item in res.get("list", []):
            name = item.get("account_nm", "")
            val = parse_amount(item.get("thstrm_amount"))

            if "영업이익" in name and not op_income:
                op_income = val
            elif ("매출액" in name or "수익" in name or "매출" in name) and not revenue:
                revenue = val

        data.append((year, op_income, revenue))

    # 데이터프레임 구성 및 계산
    df = pd.DataFrame(data, columns=["연도", "영업이익", "매출액"])
    df["영업이익률 (%)"] = df.apply(
        lambda row: round(row["영업이익"] / row["매출액"] * 100, 2)
        if pd.notnull(row["영업이익"]) and pd.notnull(row["매출액"]) and row["매출액"] != 0
        else None,
        axis=1
    )

    # ✅ CSV 저장
    os.makedirs(save_dir_csv, exist_ok=True)
    csv_filename = os.path.join(save_dir_csv, f"{corp_name}_영업이익률.csv")
    df.to_csv(csv_filename, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {csv_filename}")

    # ✅ 그래프 시각화 및 저장
    plt.figure(figsize=(10, 5))
    plt.plot(df["연도"], df["영업이익률 (%)"], marker="o", color="orange")
    plt.title(f"{corp_name} 영업이익률 추이", fontsize=14)
    plt.xlabel("연도")
    plt.ylabel("영업이익률 (%)")
    plt.grid(True)
    plt.tight_layout()

    os.makedirs(save_dir_png, exist_ok=True)
    png_filename = os.path.join(save_dir_png, f"{corp_name}_영업이익률.png")
    plt.savefig(png_filename)
    print(f"🖼️ 그래프 이미지 저장 완료: {png_filename}")

    plt.show()
    return df

for corp in CorpList:
    try:
        generate_operating_margin_df(corp)
    except Exception as e:
        print(f"⚠️ 오류 발생 - {corp}: {e}")


# 부채비율

In [ ]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt

def generate_debt_ratio_df(
    corp_name,
    save_dir_csv=r"C:\Users\1\Desktop\Dart\부채비율csv",
    save_dir_png=r"C:\Users\1\Desktop\Dart\부채비율plot"
):
    print(f"📉 {corp_name} 부채비율 계산 중...")

    corp_code_dict = get_corp_code_dict()
    corp_code = corp_code_dict.get(corp_name)
    if not corp_code:
        print(f"❌ 기업 코드 없음: {corp_name}")
        return None

    data = []
    for year in YEARS:
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": "11011",
            "fs_div": "CFS"
        }

        try:
            res = requests.get(url, params=params, timeout=10).json()
        except:
            data.append((year, None, None))
            continue

        total_liabilities = None
        total_equity = None

        # ✅ 자산/부채/자본 추출 키워드
        debt_keywords = ["부채총계", "총부채"]
        equity_keywords = ["자본총계", "지배기업 소유주지분", "지배기업지분", "총자본"]

        for item in res.get("list", []):
            name = item.get("account_nm", "")
            val = parse_amount(item.get("thstrm_amount"))

            # 부채총계
            if any(kw in name for kw in debt_keywords) and not total_liabilities:
                total_liabilities = val

            # 자본총계 또는 대체 항목
            if any(kw in name for kw in equity_keywords) and not total_equity:
                total_equity = val

        data.append((year, total_liabilities, total_equity))

    # 📊 DataFrame 구성
    df = pd.DataFrame(data, columns=["연도", "총부채", "자본총계"])
    df["부채비율 (%)"] = df.apply(
        lambda row: round(row["총부채"] / row["자본총계"] * 100, 2)
        if pd.notnull(row["총부채"]) and pd.notnull(row["자본총계"]) and row["자본총계"] != 0
        else None,
        axis=1
    )

    # ✅ CSV 저장
    os.makedirs(save_dir_csv, exist_ok=True)
    csv_filename = os.path.join(save_dir_csv, f"{corp_name}_부채비율.csv")
    df.to_csv(csv_filename, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {csv_filename}")

    # ✅ 그래프 저장
    os.makedirs(save_dir_png, exist_ok=True)
    plt.figure(figsize=(10, 5))
    plt.plot(df["연도"], df["부채비율 (%)"], marker="o", color="red")
    plt.title(f"{corp_name} 부채비율 추이")
    plt.xlabel("연도")
    plt.ylabel("부채비율 (%)")
    plt.grid(True)
    plt.tight_layout()

    png_filename = os.path.join(save_dir_png, f"{corp_name}_부채비율.png")
    plt.savefig(png_filename)
    print(f"🖼️ 그래프 이미지 저장 완료: {png_filename}")

    plt.show()
    return df


# 유동비율

In [ ]:
def generate_current_ratio_df(
    corp_name,
    save_dir_csv=r"C:\Users\1\Desktop\Dart\유동비율csv",
    save_dir_png=r"C:\Users\1\Desktop\Dart\유동비율plot"
):
    print(f"📘 {corp_name} 유동비율 계산 중...")

    corp_code_dict = get_corp_code_dict()
    corp_code = corp_code_dict.get(corp_name)
    if not corp_code:
        print(f"❌ 기업 코드 없음: {corp_name}")
        return None

    data = []
    for year in YEARS:
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
        params = {
            "crtfc_key": API_KEY,
            "corp_code": corp_code,
            "bsns_year": year,
            "reprt_code": "11011",  # 사업보고서
            "fs_div": "CFS"
        }

        try:
            res = requests.get(url, params=params, timeout=10).json()
        except:
            data.append((year, None, None, None))
            continue

        current_assets = None
        current_liabilities = None

        for item in res.get("list", []):
            name = item.get("account_nm", "")
            val = parse_amount(item.get("thstrm_amount"))

            if "유동자산" in name and not current_assets:
                current_assets = val
            elif "유동부채" in name and not current_liabilities:
                current_liabilities = val

        current_ratio = (
            round(current_assets / current_liabilities * 100, 2)
            if current_assets and current_liabilities and current_liabilities != 0
            else None
        )

        data.append((year, current_assets, current_liabilities, current_ratio))

    df = pd.DataFrame(data, columns=["연도", "유동자산", "유동부채", "유동비율 (%)"])

    # ✅ CSV 저장
    os.makedirs(save_dir_csv, exist_ok=True)
    csv_path = os.path.join(save_dir_csv, f"{corp_name}_유동비율.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {csv_path}")

    # ✅ 그래프 저장
    os.makedirs(save_dir_png, exist_ok=True)
    plt.figure(figsize=(10, 5))
    plt.plot(df["연도"], df["유동비율 (%)"], marker="o", color="teal")
    plt.title(f"{corp_name} 유동비율 추이", fontsize=14)
    plt.xlabel("연도")
    plt.ylabel("유동비율 (%)")
    plt.grid(True)
    plt.tight_layout()

    png_path = os.path.join(save_dir_png, f"{corp_name}_유동비율.png")
    plt.savefig(png_path)
    print(f"🖼️ 그래프 저장 완료: {png_path}")
    plt.show()

    return df


# 평균 급여 + 직원 수

In [ ]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

API_KEY = '' #api_key 추가
REPRT_CODE = '11011'
YEARS = ['2023', '2022', '2021']
SAVE_DIR = 'dart_charts'

# ✅ 1. DART corp_code.zip 파싱
def load_corp_codes():
    print("📦 corp_code 목록 다운로드 중...")
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={API_KEY}"
    response = requests.get(url)
    if response.status_code != 200:
        print("❌ 요청 실패: status =", response.status_code)
        return {}

    with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        with zf.open('CORPCODE.xml') as xml_file:
            tree = ET.parse(xml_file)
            root = tree.getroot()

    corp_map = {}
    for child in root.findall('list'):
        name = child.find('corp_name').text.strip()
        code = child.find('corp_code').text.strip()
        corp_map[name] = code

    print(f"✅ 총 기업 수: {len(corp_map):,}개")
    return corp_map

# ✅ 2. 직원현황 API
def fetch_emp_status(corp_code, year):
    url = "https://opendart.fss.or.kr/api/empSttus.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': year,
        'reprt_code': REPRT_CODE
    }
    res = requests.get(url, params=params)
    return res.json()

# ✅ 3. 그래프 생성 및 저장
def plot_and_save(df, corp_name):
    df = df.groupby('연도')[['정규직 수', '계약직 수', '총인원 수', '총급여액(천원)']].sum().reset_index()
    df['평균급여'] = df['총급여액(천원)'] / df['총인원 수']

    print(f"📊 {corp_name} - 평균급여 계산 결과:\n{df[['연도', '평균급여']]}")

    plt.figure(figsize=(10, 6))
    plt.plot(df['연도'], df['정규직 수'], marker='o', label='Regular')
    plt.plot(df['연도'], df['계약직 수'], marker='o', label='Contract')
    plt.plot(df['연도'], df['총인원 수'], marker='o', label='Total Employees')
    plt.plot(df['연도'], df['총급여액(천원)'] / 1_000_000_000, marker='o', label='Total Salary (조 KRW)')

    plt.title(f'{corp_name} Employment & Salary Trend')
    plt.xlabel('Year')
    plt.ylabel('Count / Salary (조 KRW)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    os.makedirs(SAVE_DIR, exist_ok=True)
    filepath = f"{SAVE_DIR}/{corp_name}_chart.png"
    plt.savefig(filepath, dpi=300)
    plt.close()
    print(f"[✅ 저장 완료] {filepath}\n")

# ✅ 4. 메인 실행
if __name__ == "__main__":
    user_input = input("기업명을 ,(콤마)로 구분해서 입력하세요 (예: 삼성전자,카카오,네이버):\n")
    customer_input_names = [name.strip() for name in user_input.split(',') if name.strip()]
    print(f"🎯 입력된 기업 수: {len(customer_input_names)}개")

    corp_map = load_corp_codes()

    for corp_name in customer_input_names:
        print(f"\n🔍 처리 중: {corp_name}")
        corp_code = corp_map.get(corp_name)
        if not corp_code:
            print(f"❌ corp_code 미발견 → '{corp_name}'은 등록되지 않았습니다.")
            continue

        records = []
        for year in YEARS:
            print(f"📅 {year} 데이터 조회 중...")
            data = fetch_emp_status(corp_code, year)
            time.sleep(0.3)
            if data.get('status') == '000' and data.get('list'):
                for row in data['list']:
                    records.append({
                        '연도': int(year),
                        '정규직 수': row.get('rgllbr_co'),
                        '계약직 수': row.get('cnttk_co'),
                        '총인원 수': row.get('sm'),
                        '총급여액(천원)': row.get('fyer_salary_totamt')
                    })
            else:
                print(f"⚠️ {year}년도 공시 없음 or 오류 메시지: {data.get('message')}")

        df = pd.DataFrame(records)
        if df.empty:
            print(f"⚠️ {corp_name} - 사용 가능한 직원 공시 데이터 없음")
            continue

        for col in ['정규직 수', '계약직 수', '총인원 수', '총급여액(천원)']:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').replace('-', ''), errors='coerce')

        print(f"📄 {corp_name} 정제된 데이터 미리보기:\n{df.head()}")
        plot_and_save(df, corp_name)


# R&D

In [ ]:
import requests, zipfile, io
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import os

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

API_KEY = "845166f5401aeebbed295c86b5f47621b51f520a"
YEARS = [str(y) for y in range(2015, 2024)]

def get_corp_code_dict(api_key=API_KEY):
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    res = requests.get(url, params={"crtfc_key": api_key})
    with zipfile.ZipFile(io.BytesIO(res.content)) as zf:
        zf.extractall()
    tree = ET.parse("CORPCODE.xml")
    root = tree.getroot()
    return {
        el.find("corp_name").text.strip(): el.find("corp_code").text.strip()
        for el in root.findall("list")
    }

def parse_amount(s):
    if not s or s in ["", "-"]:
        return None
    return int(s.replace(",", "").replace(" ", "").strip())

def get_financial_data(corp_code, year):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",
        "fs_div": "CFS"
    }
    res = requests.get(url, params=params).json()

    rd_amt = None
    sales_amt = None
    fallback_rd = None
    debug_accounts = []

    for item in res.get("list", []):
        account_nm = item.get("account_nm", "")
        debug_accounts.append(account_nm)

        if ("연구" in account_nm and "비" in account_nm) or "개발비" in account_nm:
            rd_amt = parse_amount(item.get("thstrm_amount"))
        elif "무형자산의 취득" in account_nm:
            fallback_rd = parse_amount(item.get("thstrm_amount"))
        elif "매출" in account_nm or "수익" in account_nm or "영업수익" in account_nm:
            if not sales_amt:
                sales_amt = parse_amount(item.get("thstrm_amount"))

    return rd_amt, fallback_rd, sales_amt, debug_accounts

def generate_rd_df_and_plot(corp_name, save_dir="."):
    print(f"🔍 {corp_name} 기업 데이터 수집 중...")
    corp_code_dict = get_corp_code_dict()
    corp_code = corp_code_dict.get(corp_name)

    if not corp_code:
        print(f"❌ {corp_name}의 corp_code를 찾을 수 없습니다.")
        return None

    rows = []
    for year in YEARS:
        rd_amt, fallback_rd, sales_amt, debug = get_financial_data(corp_code, year)
        used = "정확" if rd_amt else "무형자산 추정" if fallback_rd else "없음"
        final_rd = rd_amt if rd_amt else fallback_rd
        ratio = round(final_rd / sales_amt * 100, 2) if final_rd and sales_amt else None

        rows.append({
            "기업명": corp_name,
            "연도": year,
            "R&D 비용": final_rd,
            "매출액": sales_amt,
            "R&D 비중 (%)": ratio,
            "추출방법": used
        })

    df = pd.DataFrame(rows)

    # ✅ CSV 저장
    filename = os.path.join(save_dir, f"{corp_name}_R&D_비중.csv")
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"📁 CSV 저장 완료: {filename}")

    # ✅ 시각화
    plt.figure(figsize=(10, 5))
    plt.plot(df["연도"], df["R&D 비중 (%)"], marker="o", linestyle='-', label="R&D 비중 (%)")
    plt.title(f"{corp_name} 연도별 R&D 비중", fontsize=14)
    plt.xlabel("연도")
    plt.ylabel("R&D 비중 (%)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return df


# R&D 증감률

In [ ]:
# 전체 로직 (R&D 증감률)

import requests, zipfile, io
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import os

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

API_KEY = "845166f5401aeebbed295c86b5f47621b51f520a"
YEARS = [str(y) for y in range(2015, 2024)]

def get_corp_code_dict(api_key=API_KEY):
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    res = requests.get(url, params={"crtfc_key": api_key})
    with zipfile.ZipFile(io.BytesIO(res.content)) as zf:
        zf.extractall()
    tree = ET.parse("CORPCODE.xml")
    root = tree.getroot()
    return {
        el.find("corp_name").text.strip(): el.find("corp_code").text.strip()
        for el in root.findall("list")
    }

def parse_amount(s):
    if not s or s in ["", "-"]:
        return None
    return int(s.replace(",", "").replace(" ", "").strip())

def get_rd_amount(corp_code, year):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": API_KEY,
        "corp_code": corp_code,
        "bsns_year": year,
        "reprt_code": "11011",
        "fs_div": "CFS"
    }
    try:
        res = requests.get(url, params=params, timeout=10).json()
    except:
        return None

    rd_keywords = [
        "연구개발", "연구개발비", "연구개발비용", "연구비", "기술개발비",
        "개발비", "R&D", "R&D비용", "연구개발 관련 비용", "연구개발 투자"
    ]

    for item in res.get("list", []):
        account_nm = item.get("account_nm", "")
        if any(kw in account_nm for kw in rd_keywords):
            return parse_amount(item.get("thstrm_amount"))
        if "무형자산의 취득" in account_nm:
            return parse_amount(item.get("thstrm_amount"))  # fallback

    return None

def generate_rd_growth_df(
    corp_name,
    save_dir_csv=r"C:\Users\a\OneDrive\바탕 화면\실전프로젝트\DartAPI\data\기업별 R&D 증감률\csv",
    save_dir_png=r"C:\Users\a\OneDrive\바탕 화면\실전프로젝트\DartAPI\data\기업별 R&D 증감률\plot"
):
    print(f"📈 {corp_name} R&D 증감률 계산 중...")

    # 기업 코드 조회
    corp_code_dict = get_corp_code_dict()
    corp_code = corp_code_dict.get(corp_name)
    if not corp_code:
        print(f"❌ 기업 코드 없음: {corp_name}")
        return None

    # 데이터 수집
    data = []
    for year in YEARS:
        rd_amt = get_rd_amount(corp_code, year)
        data.append((year, rd_amt))

    df = pd.DataFrame(data, columns=["연도", "R&D 비용"])
    df["R&D 증감률 (%)"] = df["R&D 비용"].pct_change().apply(
        lambda x: round(x * 100, 2) if pd.notnull(x) else None
    )

    # ✅ CSV 저장
    os.makedirs(save_dir_csv, exist_ok=True)
    csv_filename = os.path.join(save_dir_csv, f"{corp_name}_R&D_증감률.csv")
    df.to_csv(csv_filename, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {csv_filename}")

    # ✅ 그래프 시각화 및 PNG 저장
    plt.figure(figsize=(10, 5))
    plt.clf()
    plt.plot(df["연도"], df["R&D 증감률 (%)"], marker="o", color="blue")
    plt.title(f"{corp_name} R&D 투자 증감률", fontsize=14)
    plt.xlabel("연도")
    plt.ylabel("증감률 (%)")
    plt.grid(True)
    plt.tight_layout()

    os.makedirs(save_dir_png, exist_ok=True)
    png_filename = os.path.join(save_dir_png, f"{corp_name}_R&D_증감률.png")
    plt.savefig(png_filename)
    print(f"🖼️ 그래프 이미지 저장 완료: {png_filename}")

    plt.show()
    return df

